In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

Будем решать задачу на определение дату следующего ТО и пробега на эту дату по некоторым характеристикам автомобилей. Для начала загрузим обучающие и тестовые данные.

In [2]:
train_data = pd.read_csv('train.csv', delimiter=';', index_col=0)
test_data = pd.read_csv('test.csv', delimiter=';', index_col=0)

## Первичная обработка

Переименуем названия колонок, чтобы было проще и быстрее к ним обращаться.

In [3]:
columns = ['model', 'brand', 'vin', 'visit_mileage', 'year', 'visit_date', 'service_interval', 'is_service']
train_data.columns = columns
test_data.columns = columns

Посмотрим некоторую информацию таблицы и вообще что она из себя представляет.

In [4]:
train_data

,model,brand,vin,visit_mileage,year,visit_date,service_interval,is_service
0,LADA Largus Caravan (7м),LADA,27837,29 644,01.01.2014 0:00:00,03.08.2015 0:00:00,15 000,Нет
1,LADA Largus Caravan (7м),LADA,27837,29 644,01.01.2014 0:00:00,06.08.2015 0:00:00,15 000,Да
2,LADA Largus Caravan (7м),LADA,27837,62 152,01.01.2014 0:00:00,28.04.2016 0:00:00,15 000,Нет
3,LADA Largus Caravan (7м),LADA,27837,104 743,01.01.2014 0:00:00,09.12.2016 0:00:00,15 000,Да
4,УАЗ Patriot 3163,УАЗ,28253,29 979,01.01.2016 0:00:00,27.01.2018 0:00:00,15 000,Да
...,...,...,...,...,...,...,...,...
109231,Toyota Corolla X E140/E150,TOYOTA,10244,85 847,01.01.2011 0:00:00,12.04.2016 0:00:00,10 000,Нет
109232,Toyota Corolla X E140/E150,TOYOTA,10244,89 316,01.01.2011 0:00:00,11.06.2016 0:00:00,10 000,Нет
109233,Toyota Corolla X E140/E150,TOYOTA,10244,89 316,01.01.2011 0:00:00,12.06.2016 0:00:00,10 000,Нет
109234,Hyundai ix35 I,Hyundai,15476,25 750,01.01.2014 0:00:00,29.07.2017 0:00:00,15 000,Да


In [5]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 109236 entries, 0 to 109235
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   model             109236 non-null  object
 1   brand             109236 non-null  object
 2   vin               109236 non-null  int64 
 3   visit_mileage     108832 non-null  object
 4   year              109236 non-null  object
 5   visit_date        109236 non-null  object
 6   service_interval  109236 non-null  object
 7   is_service        109236 non-null  object
dtypes: int64(1), object(7)
memory usage: 7.5+ MB


Так как нам надо предсказывать дату следующего ТО, то целесообразно будет убрать записи, которые не являются посещениями по ТО, потому что они не помогут в обучении.

In [6]:
train_data = train_data[train_data['is_service'] == 'Да'].drop(columns=['is_service']).reset_index(drop=True)
test_data = test_data.drop(columns=['is_service']).reset_index(drop=True)

Как видно, почти все признаки представлены как object. Давайте преобразуем их в соответствующие форматы. Сперва преобразуем категориальные признаки.

In [7]:
categorical_cols = ['model', 'brand']

train_data[categorical_cols] = train_data[categorical_cols].astype('category')
test_data[categorical_cols] = test_data[categorical_cols].astype('category')

Далее с датами. У признака year, думаю, целесообразно оставить только год.

In [8]:
train_data['visit_date'] = pd.to_datetime(train_data['visit_date'], format='%d.%m.%Y %H:%M:%S', errors='coerce')
train_data['year'] = pd.to_datetime(train_data['year'], format='%d.%m.%Y %H:%M:%S', errors='coerce').dt.year

test_data['visit_date'] = pd.to_datetime(test_data['visit_date'], format='%d.%m.%Y %H:%M:%S', errors='coerce')
test_data['year'] = pd.to_datetime(test_data['year'], format='%d.%m.%Y %H:%M:%S', errors='coerce').dt.year

Числовые признаки представлены как строка числа, разделённого пробелами по разрядам. Уберём их и преобразуем в числа.

In [9]:
train_data['visit_mileage'] = train_data['visit_mileage'].str.replace(r'\s+', '', regex=True)
train_data['visit_mileage'] = pd.to_numeric(train_data['visit_mileage'])

train_data['service_interval'] = train_data['service_interval'].str.replace(r'\s+', '', regex=True)
train_data['service_interval'] = pd.to_numeric(train_data['service_interval'])

test_data['visit_mileage'] = test_data['visit_mileage'].str.replace(r'\s+', '', regex=True)
test_data['visit_mileage'] = pd.to_numeric(test_data['visit_mileage'])

test_data['service_interval'] = test_data['service_interval'].str.replace(r'\s+', '', regex=True)
test_data['service_interval'] = pd.to_numeric(test_data['service_interval'])

Наконец посмотрим на результаты изменений.

In [10]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41603 entries, 0 to 41602
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   model             41603 non-null  category      
 1   brand             41603 non-null  category      
 2   vin               41603 non-null  int64         
 3   visit_mileage     41592 non-null  float64       
 4   year              41489 non-null  float64       
 5   visit_date        41603 non-null  datetime64[ns]
 6   service_interval  41603 non-null  int64         
dtypes: category(2), datetime64[ns](1), float64(2), int64(2)
memory usage: 1.7 MB


## Обработка пропущенных значений

Посмотрим есть ли пропущенные значения.

In [11]:
train_data.isna().sum()

model                 0
brand                 0
vin                   0
visit_mileage        11
year                114
visit_date            0
service_interval      0
dtype: int64

In [12]:
test_data.isna().sum()

model                0
brand                0
vin                  0
visit_mileage        0
year                25
visit_date           0
service_interval     0
dtype: int64

В столбцах visit_mileage и year есть пропущенные значения. Заполним их медианой.

In [13]:
from sklearn.impute import SimpleImputer

col_with_missing_values = ['year', 'visit_mileage']

imp = SimpleImputer(strategy='median')
imp.fit(train_data[col_with_missing_values])

train_data[col_with_missing_values] = imp.transform(train_data[col_with_missing_values])
test_data[col_with_missing_values] = imp.transform(test_data[col_with_missing_values])

## Создание новых признаков

Добавим признаки: среднее время между визитами и средний пробег между визитами. Они будут для каждой уникальной машины (по VIN-номеру). Для этого сначала для каждой записи вычислим время и пробег с последнего визита.

In [14]:
train_data = train_data.sort_values(['vin', 'visit_date'])
test_data = test_data.sort_values('vin')

train_data['days_since_service']=train_data.groupby('vin')['visit_date'].diff().dt.days
train_data['mileage_since_service'] = train_data.groupby('vin')['visit_mileage'].diff()

Для записей, у которых указана дата первого визита по ТО, сейчас этих неопределены. Заполним их нулями.

In [15]:
train_data['days_since_service'] = train_data['days_since_service'].fillna(0)
train_data['mileage_since_service'] = train_data['mileage_since_service'].fillna(0)

Перейдем к средним значениям

In [16]:
vin_stats = train_data.groupby('vin').agg({
    'days_since_service': 'mean',
    'mileage_since_service': 'mean',
}).rename(columns={
    'days_since_service': 'avg_days',
    'mileage_since_service': 'avg_mileage',
}).reset_index()

Объеденяем

In [17]:
train_data = train_data.merge(vin_stats, on='vin', how='left')
test_data = test_data.merge(vin_stats, on='vin', how='right')

Удалим столбцы со днями и пробегом с последнего визита. Вообще они могли бы помочь в тренировке модели, но их нет в тестовой таблице.

In [18]:
train_data = train_data.drop(columns=['days_since_service', 'mileage_since_service'])

Добавим также стобцы с предыдущей датой посещения ТО и предыдущего пробега.

In [19]:
train_data['prev_visit_date'] = train_data.groupby('vin')['visit_date'].shift(1)
train_data['prev_visit_mileage'] = train_data.groupby('vin')['visit_mileage'].shift(1)

In [20]:
test_data['prev_visit_date'] = train_data.groupby('vin')['visit_date'].max().to_numpy()
test_data['prev_visit_mileage'] = train_data.groupby('vin')['visit_mileage'].max().to_numpy()

Заполним первые значения: для даты - годом выпуска автомобиля, для пробега - нулём.

In [21]:
train_data['prev_visit_date'] = train_data['prev_visit_date'].fillna(train_data['year'].astype(int).astype(str) + '-01-01')
train_data['prev_visit_mileage'] = train_data['prev_visit_mileage'].fillna(0)

## Обработка категориальных признаков

Имеем два категориальных признака: модель и бренд автомобиля. Разберемся сначала с брендом, посмотрим сколько там уникальных значений.

In [22]:
train_data.brand.unique()

['TOYOTA', 'LEXUS', 'MERCEDES-BENZ', 'MITSUBISHI', 'INFINITI', ..., 'CHEVROLET NIVA', 'ISUZU', 'УАЗ', 'GENESIS', 'KIA']
Length: 20
Categories (20, object): ['BMW', 'CHEVROLET NIVA', 'Citroen', 'FORD', ..., 'Skoda', 'TOYOTA', 'ГАЗ', 'УАЗ']

Всего 20 уникальных значений. Это немного, поэтому будем использовать метод One-Hot Encoding.

In [23]:
train_data = pd.get_dummies(train_data, columns=['brand'])
test_data = pd.get_dummies(test_data, columns=['brand'])

Посмотрим признак model

In [24]:
train_data.model.unique()

['Toyota Estima', 'Lexus RX III 350', 'Lexus RX II 330', 'Mercedes-Benz GL-klasse X164', 'Mercedes-Benz M-klasse AMG W166', ..., 'KIA Rio IV', 'Hyundai Solaris Sedan', 'Hyundai Solaris Hatchback', 'Hyundai Creta', 'Hyundai Solaris HCR']
Length: 411
Categories (411, object): ['172411', '38787-0000010-51', '38787-0000010-94', '38787-92', ..., 'УАЗ-315148', 'УАЗ-390945', 'УАЗ-390995', 'УАЗ-39623-СГР']

Их достаточно много, поэтому используем метод Label Encoding

In [25]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(train_data['model'])

train_data['model'] = le.transform(train_data['model'])
test_data['model'] = le.transform(test_data['model'])

Столбец с номерами VIN также можно рассматривать как категориальный признак, посмотриим сколько уникальных значений.

In [26]:
train_data.vin.unique()

array([   35,    41,    42, ..., 46272, 46295, 46313])

Для обработки признака vin был выбран метод Frequency Encoding, так как он позволяет сохранить информацию о частоте появления каждого уникального значения без увеличения размерности данных. Это обеспечивает баланс между сохранением важной информации и вычислительной эффективностью модели.

In [27]:
vin_frequency = train_data.vin.value_counts().to_dict()
train_data.vin = train_data.vin.map(vin_frequency)
test_data.vin = test_data.vin.map(vin_frequency)

## Преобразование временных рядов

Преобразование временных рядов в числовой формат необходимо, поскольку модели машинного обучения не могут напрямую работать с датами. Это преобразование позволяет унифицировать данные и сохранить информацию о временной последовательности событий, что важно для предсказания даты следующего ТО и пробега автомобиля.

In [28]:
train_data['visit_date'] = (train_data['visit_date'] - pd.Timestamp('1970-01-01')) // pd.Timedelta('1D')
train_data['prev_visit_date'] = (train_data['prev_visit_date'] - pd.Timestamp('1970-01-01')) // pd.Timedelta('1D')

test_data['visit_date'] = (test_data['visit_date'] - pd.Timestamp('1970-01-01')) // pd.Timedelta('1D')
test_data['prev_visit_date'] = (test_data['prev_visit_date'] - pd.Timestamp('1970-01-01')) // pd.Timedelta('1D')

## Масштабирование количественных признаков

Выполним стандартизацию количественных признаков

In [29]:
# from sklearn.preprocessing import StandardScaler

# quantitative_features = ['visit_mileage', 'year', 'service_interval', 'avg_days', 'avg_mileage', 'visit_date', 'prev_visit_date', 'prev_visit_mileage']
# scaler = StandardScaler()
# scaler.fit(train_data[quantitative_features])

# train_data[quantitative_features] = scaler.transform(train_data[quantitative_features])
# test_data[quantitative_features] = scaler.transform(test_data[quantitative_features])

## Разделение на признаки и целевые переменные

Задачи прогнозирования даты следующего ТО и пробега автомобиля будут решаться как две отдельные задачи регрессии.

In [30]:
X_train = train_data.drop(columns=['visit_mileage', 'visit_date'])
y_train_mileage = train_data['visit_mileage']
y_train_date = train_data['visit_date']

X_test = test_data.drop(columns=['visit_mileage', 'visit_date'])
y_test_mileage = test_data['visit_mileage']
y_test_date = test_data['visit_date']

## Обучение модели

Метод Random Forest был выбран для решения задачи предсказания даты следующего ТО и пробега автомобиля из-за его устойчивости к переобучению, способности работать с числовыми и категориальными признаками, а также  не требует масштабирования признаков, что упрощает предварительную обработку данных.

In [39]:
from sklearn.ensemble import RandomForestRegressor

model_date = RandomForestRegressor(random_state=42, n_jobs=-1)
model_date.fit(X_train, y_train_date)

RandomForestRegressor(n_jobs=-1, random_state=42)

In [42]:
model_mileage = RandomForestRegressor(random_state=42, n_jobs=-1)
model_mileage.fit(X_train, y_train_mileage)

RandomForestRegressor(n_jobs=-1, random_state=42)

## Предсказания на тестовом наборе и оценка качества модели

Получим предсказания для тестового набора

In [43]:
y_pred_date = model_date.predict(X_test)
y_pred_mileage = model_mileage.predict(X_test)

Оценим качество модели с помощью метрик $R^2$ и *MAE*. *MAE* показывает среднюю абсолютную ошибку, что помогает понять реальную величину ошибок модели, тогда как $R^2$ показывает количество отклонений в прогнозах (у идеального решающего правило равно единице).

In [44]:
from sklearn.metrics import mean_absolute_error, r2_score

mae_date = mean_absolute_error(y_test_date, y_pred_date)
r2_date = r2_score(y_test_date, y_pred_date)

mae_mileage = mean_absolute_error(y_test_mileage, y_pred_mileage)
r2_mileage = r2_score(y_test_mileage, y_pred_mileage)

print(f'MAE для даты: {mae_date}')
print(f'R^2 для даты: {r2_date}')

print(f'MAE для пробега: {mae_mileage}')
print(f'R^2 для пробега: {r2_mileage}')

MAE для даты: 85.98538843290703
R^2 для даты: 0.9557383905883697
MAE для пробега: 3707.2805311128077
R^2 для пробега: 0.9956905462313719


Средняя абсолютная ошибка предсказания даты следующего ТО составляет около 86 дней. Это означает, что в среднем предсказанные даты отличаются от фактических на 86 дней.

Средняя абсолютная ошибка предсказания пробега составляет около 3707 километров. Это означает, что в среднем предсказанные пробеги отличаются от фактических на 3707 километров.

Значение $R^2$ близко к 1 (0.96 и 0.99), что означает, что модель очень хорошо объясняет вариацию даты следующего Т и пробега. Это говорит о высоком качестве модели в плане объяснения данных.